In [1]:
import pandas as pd
import numpy as np
import openpyxl
import sys
import os
from pathlib import Path

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\utils")

from players_season_builder import *

In [2]:
folder_path = r"\Users\sebas\Desktop\UChicago - Q6\Sports Analytics\sports_analytics_project\data\Big Data Ball - Game" 
file_type, sheet_name = "xlsx" , 0
data_source = "BDB-GAME"


In [10]:
temp_df = combine_xl_data(folder_path, file_type, sheet_name, data_source) 

In [11]:
temp_df['TEAMS'].unique()

array(['Chicago', 'Miami', 'Phoenix', 'LA Lakers', 'Indiana', 'Charlotte',
       'Orlando', 'Atlanta', 'Philadelphia', 'New Orleans', 'Boston',
       'Toronto', 'New Jersey', 'Washington', 'Cleveland', 'Milwaukee',
       'Detroit', 'New York', 'Memphis', 'Sacramento', 'Minnesota',
       'Houston', 'Utah', 'Portland', 'Seattle', 'Golden State',
       'LA Clippers', 'San Antonio', 'Dallas', 'Denver', 'Oklahoma City',
       'Brooklyn'], dtype=object)

Handle binary variables
- Rest Days:    
    '3+', 'B2B', 1, 2, '3IN4', '3IN4-B2B', '4IN5-B2B', '4IN5-B2B2B', '4IN5', '5IN6', 'B2B2B', '3IN4-B2B2B', '5IN6-B2B2B', '1', '2' '5IN5-B2B2B'

    Assumptions: 
        B2B = 0 days of rest
        Number of Days: 0, 1, 2, 3+
        Density of Games: How do we want to consider density

    # The more accurate method will be comparing dates and manually calculating 

- Applicable Data:

    Filter for "Regular Season"


In [20]:
# Filter for Regular Season
nba_regular_season = temp_df[temp_df['DATASET'].str.contains("Season")]
nba_regular_season['DATASET'] = nba_regular_season['DATASET'].str.replace(
    'NBA 2018-2019 Regular Season',
    '2018-2019 Regular Season'
)
nba_regular_season['DATASET'] = nba_regular_season['DATASET'].str.replace(
    'NBA 2019-2020 Regular Season',
    '2019-2020 Regular Season'
)
# Filter for playoffs
nba_playoffs = temp_df[temp_df['DATASET'].str.contains("Playoffs")]
nba_regular_season['DATASET'] = nba_regular_season['DATASET'].str.replace(
    'NBA 2020 Playoffs', '2020 Playoffs')
nba_regular_season['DATASET'] = nba_regular_season['DATASET'].str.replace(
    'NBA 2019 Playoffs','2019 Playoffs')


<ipython-input-20-2007e87bbf61>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_regular_season['DATASET'] = nba_regular_season['DATASET'].str.replace(
<ipython-input-20-2007e87bbf61>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_regular_season['DATASET'] = nba_regular_season['DATASET'].str.replace(
<ipython-input-20-2007e87bbf61>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [21]:
# Address Types
nba_regular_season['REST DAYS'] = nba_regular_season['REST DAYS'].astype('string')
nba_regular_season['DATE'] = pd.to_datetime(
    nba_regular_season['DATE']
    )

<ipython-input-21-0f00ec012475>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_regular_season['REST DAYS'] = nba_regular_season['REST DAYS'].astype('string')
<ipython-input-21-0f00ec012475>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_regular_season['DATE'] = pd.to_datetime(


In [22]:
nba_regular_season['TEAMS'].unique()

array(['Chicago', 'Miami', 'Phoenix', 'LA Lakers', 'Indiana', 'Charlotte',
       'Orlando', 'Atlanta', 'Philadelphia', 'New Orleans', 'Boston',
       'Toronto', 'New Jersey', 'Washington', 'Cleveland', 'Milwaukee',
       'Detroit', 'New York', 'Memphis', 'Sacramento', 'Minnesota',
       'Houston', 'Utah', 'Portland', 'Seattle', 'Golden State',
       'LA Clippers', 'San Antonio', 'Dallas', 'Denver', 'Oklahoma City',
       'Brooklyn'], dtype=object)

In [65]:
# Handle Rest Day - Recalculation
test = list()
for season in nba_regular_season['DATASET'].unique():
    for team in list(nba_regular_season[nba_regular_season['DATASET'] == season]['TEAMS'].unique()):
        temp_season = nba_regular_season.loc[nba_regular_season['DATASET'].eq(season) & nba_regular_season['TEAMS'].eq(team) ].sort_values(by=['TEAMS', 'DATE'])
        temp_season['DATE-DIFF'] = temp_season['DATE'].diff().dt.days
        temp_season['H/R CHANGE'] = temp_season['VENUE'].shift(1) != temp_season['VENUE'] # H --> R or R --> H 
        # Not sure how to handle multiple Road games
        temp_season['RR VAL'] = temp_season['VENUE'].shift(1) + temp_season['VENUE']
        temp_season['RR SERIES'] = ( temp_season['VENUE'].shift(1) + temp_season['VENUE']) == "RR"
        test.append(temp_season)

nba_regular_season = pd.concat(test)
nba_regular_season['DATE-DIFF'] = nba_regular_season['DATE-DIFF'].astype(str)
nba_regular_season['DATE-DIFF'] = nba_regular_season['DATE-DIFF']+" days"



In [66]:
# Create Dummy Variables for 'REST DAYS'
nba_reg_season_dummies = pd.concat([
    nba_regular_season,
    pd.get_dummies(nba_regular_season['REST DAYS'])
], axis=1)

In [67]:
game = [ 'DATASET',       'DATE',       'VENUE',      'TEAMS',          'F',        
              'MIN',         'FG',        'FGA',         '3P',        '3PA',         
               'FT',        'FTA',         'OR',         'DR',        'TOT',          
                'A',         'PF',         'ST',         'TO',         'BL',        
              'PTS',       'POSS',       'PACE',       'OEFF',       'DEFF' ]

nba_reg_season_stats = nba_regular_season[game].groupby(['TEAMS', 'DATASET'], dropna=True).mean()
nba_reg_season_stats_home_away = nba_regular_season[game].groupby(['TEAMS', 'DATASET','VENUE'], dropna=True).mean()

In [68]:
nba_reg_dates = pd.concat([
    nba_regular_season[game].groupby(['TEAMS', 'DATASET'], dropna=True)['DATE'].min(), 
    nba_regular_season[game].groupby(['TEAMS', 'DATASET'], dropna=True)['DATE'].max(),
], axis = 1 )

nba_reg_season_dummies = pd.concat([
    nba_regular_season,
    pd.get_dummies(nba_regular_season['DATE-DIFF'])
], axis=1)

In [56]:
nba_reg_dates

DATE       DATE
TEAMS      DATASET                                       
Atlanta    2006-2007 Regular Season 2006-11-01 2007-04-17
           2007-2008 Regular Season 2007-11-02 2008-04-16
           2008-2009 Regular Season 2008-10-29 2009-04-15
           2009-2010 Regular Season 2009-10-28 2010-04-14
           2010-2011 Regular Season 2010-10-27 2011-04-13
...                                        ...        ...
Washington 2015-2016 Regular Season 2015-10-28 2016-04-13
           2016-2017 Regular Season 2016-10-27 2017-04-12
           2017-2018 Regular Season 2017-10-18 2018-04-11
           2018-2019 Regular Season 2018-10-18 2019-04-09
           2019-2020 Regular Season 2019-10-23 2020-08-13

[420 rows x 2 columns]

Season Statistics -> 

In [72]:
with pd.ExcelWriter("../data/BDB_Combo.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    nba_regular_season.to_excel(writer, sheet_name="General RS", index=False)
    nba_reg_dates.to_excel(writer, sheet_name="RS - Date", index=True)
    nba_reg_season_dummies.to_excel(writer, sheet_name= "RS - Dummies", index=False)
    nba_reg_season_stats.to_excel(writer, sheet_name="RS - Avg Season Stats", index=True)
    nba_reg_season_stats_home_away.to_excel(writer, sheet_name="RS - HomeAway Stats", index=True)